# Week 11 Group Activity

### 1. Explore Google Trends https://trends.google.com/ . Find a topic of interest to you with more than a year of trend data, and export it.


#### a. Import the data into a pandas dataframe.

#### b. Decide whether ARIMA or SARIMA makes more sense for the dataset.

#### c. Run either an ARIMA or SARIMA on it.

#### d. Explain the results.

#### e. Discuss your findings with your group and explain what trend someone else chose and why they chose the model they did.

### 2. What is a fourier transform? 

#### Perform one on y = np.cos(x/2), and both show (with code) and explain the key elements